In [ ]:
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.feature_extraction import get_graph_node_names
import rsatoolbox.rdm.calc
import rsatoolbox.data
import sys
import torch

import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np

sys.path.append("../src")

import utils  # noqa: E402


def sample_images(data_loader, n=5, plot=False):
    """Samples a specified number of images from a data loader."""
    imgs, labels = next(iter(data_loader))

    imgs_o = []
    targets = []
    for value in range(10):
        imgs_o.append(imgs[np.where(labels == value)][0:n])
        targets.append([value] * n)

    imgs = torch.cat(imgs_o, dim=0)
    targets = torch.tensor(targets).flatten()

    if plot:
        plt.imshow(
            torch.moveaxis(
                make_grid(imgs, nrow=n, padding=0, normalize=False, pad_value=0), 0, -1
            )
        )
        plt.title("Sampled Test Images (5 of each class)")
        plt.axis("off")
        plt.show()

    return imgs, targets


def extract_features(model, imgs, return_layers):
    """Extracts features from specified layers of the model."""
    if return_layers == "all":
        # Automatically get the names of all layers in the model
        return_layers, _ = get_graph_node_names(model)

    print(f"{return_layers=}")

    # Create the feature extractor
    feature_extractor = create_feature_extractor(model, return_nodes=return_layers)
    model_features = feature_extractor(imgs)

    # Re-key with simple integer indices for easier plotting and add the input layer
    final_features = {i: v for i, (k, v) in enumerate(model_features.items())}
    final_features[-1] = imgs  # Add input layer with key -1

    # Sort the dictionary by layer index to ensure the correct processing order
    final_features = dict(sorted(final_features.items()))

    return final_features


def extract_features_(model, imgs, return_layers):
    if isinstance(return_layers, str) and return_layers == "all":
        # Automatically get the names of all layers in the model
        return_layers, _ = get_graph_node_names(model)

    print(f"{return_layers=}")

    # Create the feature extractor
    feature_extractor = create_feature_extractor(model, return_nodes=return_layers)
    model_features = feature_extractor(imgs)
    return model_features


def calc_rdms(model_features, method="correlation"):
    """Calculates representational dissimilarity matrices (RDMs) for model features.

    Args:
        model_features: A dictionary where keys are layer names and values are features of the layers.
        method: The method to calculate RDMs, e.g., 'correlation'. Default is 'correlation'.

    Outputs:
        rdms: RDMs object containing dissimilarity matrices.
        rdms_dict: A dictionary with layer names as keys and their corresponding RDMs as values.
    """
    ds_list = []
    for l in range(len(model_features)):
        layer = list(model_features.keys())[l]
        feats = model_features[layer]

        if type(feats) is list:
            feats = feats[-1]

        feats = feats.cpu()

        if len(feats.shape) > 2:
            feats = feats.flatten(1)

        feats = feats.detach().numpy()
        ds = rsatoolbox.data.Dataset(feats, descriptors=dict(layer=layer))
        ds_list.append(ds)

    rdms = rsatoolbox.rdm.calc.calc_rdm(ds_list, method=method)
    rdms_dict = {
        list(model_features.keys())[i]: rdms.get_matrices()[i]
        for i in range(len(model_features))
    }

    return rdms, rdms_dict


def plot_maps(model_features, model_name):
    """Plots representational dissimilarity matrices (RDMs) across different layers."""
    fig = plt.figure(figsize=(14, 4))
    fig.suptitle(f"RDMs across layers for {model_name}")
    gs = fig.add_gridspec(1, len(model_features))
    fig.subplots_adjust(wspace=0.2, hspace=0.2)

    for l, layer in enumerate(model_features.keys()):
        map_ = np.squeeze(model_features[layer])

        if len(map_.shape) < 2:
            side_len = int(np.sqrt(map_.shape[0]))
            if side_len * side_len == map_.shape[0]:
                map_ = map_.reshape((side_len, side_len))

        if np.max(map_) > 0:
            map_ = map_ / np.max(map_)

        ax = plt.subplot(gs[0, l])
        ax_ = ax.imshow(map_, cmap="magma_r")
        ax.set_title(f"Layer: {layer}")
        ax.set_xlabel("Input Index")
        if l == 0:
            ax.set_ylabel("Input Index")

    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.92, 0.15, 0.01, 0.7])
    cbar = fig.colorbar(ax_, cax=cbar_ax)
    cbar.set_label("Dissimilarity", rotation=270, labelpad=15)

    plt.show()

In [ ]:
import models
import torch.nn as nn


def create_model(n_classes, latent_dim=84):
    """Creates a fresh instance of the CNN architecture."""
    return nn.Sequential(
        nn.Sequential(nn.Conv2d(1, 6, 5), nn.ReLU(), nn.MaxPool2d(2, 2)),
        nn.Sequential(nn.Conv2d(6, 16, 5), nn.ReLU(), nn.MaxPool2d(2)),
        nn.Sequential(nn.Flatten(), nn.Linear(16 * 5 * 5, 120), nn.ReLU()),
        nn.Sequential(nn.Linear(120, latent_dim), nn.ReLU()),
        nn.Sequential(nn.Linear(latent_dim, n_classes)),
    )


# Config
batch_size = 32
n_classes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models
bp_model = models.LeNet5(n_classes=10, latent_dim=84, act_fn="relu")
bp_model.load_state_dict(torch.load("../results/backprop-model.pth"))
bp_model.eval().to(device)

ff_model = models.FFLeNet5(n_classes=10, latent_dim=84)
ff_model.load_state_dict(torch.load("../results/ff-model.pth"))
ff_model.eval().to(device)

pc_model = create_model(n_classes=n_classes)
pc_model.load_state_dict(torch.load("../results/pc-model.pth"))
pc_model.eval().to(device)

# Load data
_, _, test_loader = utils.load_mnist_data(batch_size)
class_names = [str(i) for i in range(n_classes)]

In [ ]:
# name_map = {
#     '0.0': 'conv1',
#     '0.1': 'relu',
#     '0.2': 'pool1',
#     '1.0': 'conv2',
#     '1.1': 'relu_1',
#     '1.2': 'pool2',
#     '2.0': 'flatten',
#     '2.1': 'fc1',
#     '2.2': 'relu_2',
#     '3.0': 'fc2',
#     '3.1': 'relu_3',
#     '4.0': 'output'
# }

train_nodes, val_nodes = get_graph_node_names(bp_model)
print(f"{train_nodes=}")
print(f"{val_nodes=}\n")

train_nodes, val_nodes = get_graph_node_names(ff_model)
print(f"{train_nodes=}")
print(f"{val_nodes=}\n")

train_nodes, val_nodes = get_graph_node_names(pc_model)
print(f"{train_nodes=}")
print(f"{val_nodes=}")

In [ ]:
# FIXME: it's not returning 5 samples per class
imgs, targets = sample_images(test_loader, n=5, plot=True)

print("Extracting features (backprop)")
layer_names = ["conv1", "conv2", "conv3", "fc1", "fc2"]
# layer_names = ["pool1", "pool2", "flatten", "relu_3", "fc2"]
bp_features = extract_features(bp_model, imgs.to(device), return_layers=layer_names)

# FIXME: input[32, 1, 32, 32] to have 11 channels, but got 1 channels instead
# print("\nExtracting features (forward-forward)")
# layer_names = ["conv1.conv", "conv2.conv", "conv3.conv", "fc1.linear", "fc2.linear"]
# ff_features = extract_features(ff_model, imgs.to(device), return_layers=layer_names)

print("\nExtracting features (predictive coding)")
layer_names = [0, 1, 2, 3, 4]
pc_features = extract_features_(pc_model, imgs.to(device), return_layers=layer_names)

In [ ]:
rdms_bp, rdms_dict_bp = calc_rdms(bp_features)
plot_maps(rdms_dict_bp, "Backpropagation")

rdms_pc, rdms_dict_pc = calc_rdms(pc_features)
plot_maps(rdms_dict_pc, "Predictive Coding")